### Measuring and characterizing fairness as a notion of inclusiveness.

Certain Machine Learning models are made to perform classification tasks of samples over properties which are subjective, what means that several users of the models might judge the property of the sample differently depending on their personal experience.
The predictions of the models might contain biases towards certain types of judgements which are more common than others and consequently easier to learn, and ignore other judgements. However for the predictions to be fair towards each user of the model, they should be inclusive of all the different judgements, and possibly should be tuned to each of the users.
In this tutorial we teach how to use metrics to measure how fair as a notion of inclusiveness the models are, and how to use different characterizations of the predictions to understand where the unfairness might come from.

# Import statements

In [1]:
# Load all necessary packages
import sys
sys.path.append("../")  

#from aif360.datasets import GermanDataset
from aif360.datasets import ToxicityDataset
from aif360.metrics import InclusivenessLabelDatasetMetric

from IPython.display import Markdown, display


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import sklearn.metrics as sk_met
from sklearn.base import TransformerMixin, BaseEstimator

import numpy as np
import pickle

## Load the toxicity dataset

In [2]:
### The toxicity dataset (toxicity_annotations.tsv, toxicity_annotated_comments.tsv, toxicity_worker_demographics.tsv) should be downloaded from https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973
### and placed in the folder "data/raw/toxicity".

tox_dataset = ToxicityDataset()

c:\users\agathebalayn\documents\thesis_related\virtualenvs\aif306_v2\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
dataset_orig_train, dataset_orig_test = tox_dataset.split([0.7], shuffle=True)

In [4]:
dataset_orig_train.dataset

,index,rev_id,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,pop_label,general_split,sample_agreement,MV,average_toxicity,disaggreement_bin,annotator_ADR,popularity_percentage,comment
1264596,1533960,659564670.0,60,0,0.0,male,0.0,18-30,bachelors,male 18-30 bachelors,train,1.0,0,0.0,0,0.061404,1.0,also unfounded claim documented tribal prefere...
334830,406563,128933407.0,688,0,1.0,male,0.0,30-45,bachelors,male 30-45 bachelors,train,1.0,0,0.0,0,0.054348,1.0,call marching bell afraid revert edit frst cha...
804810,976735,348759615.0,1826,0,0.0,female,0.0,18-30,masters,female 18-30 masters,train,1.0,0,0.0,0,0.060241,1.0,withdraw 10usd bank account
253576,307735,93631183.0,2991,1,-1.0,female,1.0,30-45,masters,female 30-45 masters,train,0.7,0,0.3,1,0.123318,0.3,page overrun muslim pov pushers note number mu...
458694,556898,183869619.0,4227,0,1.0,male,1.0,18-30,bachelors,male 18-30 bachelors,train,1.0,0,0.0,0,0.029126,1.0,remember banned couple months ago really terri...
1111670,1348505,550905110.0,1095,0,0.0,female,0.0,18-30,bachelors,female 18-30 bachelors,train,1.0,0,0.0,0,0.083916,1.0,jōetsu hello ansei thank attention jōetsu regi...
405750,492759,161269832.0,2315,0,1.0,female,0.0,30-45,bachelors,female 30-45 bachelors,train,0.8,0,0.2,0,0.036866,0.8,2007 utc please learn post less garrishly cont...
1126902,1366983,561549440.0,2617,0,0.0,male,0.0,30-45,masters,male 30-45 masters,train,0.9,0,0.1,0,0.046296,0.9,bottom line let people use wikipedia settle re...
689734,837244,287371502.0,2826,0,1.0,male,0.0,18-30,professional,male 18-30 professional,train,1.0,0,0.0,0,0.048611,1.0,daphne odjig driving force behind indian group...
191357,232305,70472121.0,2623,0,0.0,male,0.0,18-30,bachelors,male 18-30 bachelors,train,1.0,0,0.0,0,0.077982,1.0,also university arts http www arts bg ac yu


## Train a model with the loaded data

In [5]:
### Usually the annotations are aggregated into the majority vote (MV).
def prepare_aggregated_data(dataset_orig_train):
    dataset_train_comments = dataset_orig_train.dataset.drop_duplicates('rev_id')
    dataset_train_comments = dataset_train_comments.drop('toxicity', 1)
    dataset_train_comments = dataset_train_comments.drop('toxicity_score', 1)
    dataset_train_comments = dataset_train_comments.drop('worker_id', 1)
    dataset_train_comments = dataset_train_comments.drop('gender', 1)
    dataset_train_comments = dataset_train_comments.drop('age_group', 1)
    dataset_train_comments = dataset_train_comments.drop('education', 1)
    dataset_train_comments = dataset_train_comments.drop('pop_label', 1)
    dataset_train_comments = dataset_train_comments.drop('english_first_language', 1)
    return dataset_train_comments

In [ ]:
dataset_train_comments = prepare_aggregated_data(dataset_orig_train)
dataset_test_comments = prepare_aggregated_data(dataset_orig_test)


In [ ]:
### The ML model is then trained on the MV labels. 

embedding_model = 0
dense_outputs = 100 
batches = 128
sequence_length = 100 

In [ ]:
dataset_train_comments

,index,rev_id,general_split,sample_agreement,MV,average_toxicity,disaggreement_bin,annotator_ADR,popularity_percentage,comment
1264596,1533960,659564670.0,train,1.000000,0,0.000000,0,0.061404,1.000000,also unfounded claim documented tribal prefere...
334830,406563,128933407.0,train,1.000000,0,0.000000,0,0.054348,1.000000,call marching bell afraid revert edit frst cha...
804810,976735,348759615.0,train,1.000000,0,0.000000,0,0.060241,1.000000,withdraw 10usd bank account
253576,307735,93631183.0,train,0.700000,0,0.300000,1,0.123318,0.300000,page overrun muslim pov pushers note number mu...
458694,556898,183869619.0,train,1.000000,0,0.000000,0,0.029126,1.000000,remember banned couple months ago really terri...
1111670,1348505,550905110.0,train,1.000000,0,0.000000,0,0.083916,1.000000,jōetsu hello ansei thank attention jōetsu regi...
405750,492759,161269832.0,train,0.800000,0,0.200000,0,0.036866,0.800000,2007 utc please learn post less garrishly cont...
1126902,1366983,561549440.0,train,0.900000,0,0.100000,0,0.046296,0.900000,bottom line let people use wikipedia settle re...
689734,837244,287371502.0,train,1.000000,0,0.000000,0,0.048611,1.000000,daphne odjig driving force behind indian group...
191357,232305,70472121.0,train,1.000000,0,0.000000,0,0.077982,1.000000,also university arts http www arts bg ac yu


In [ ]:
class DataFrameColumnExtracter(BaseEstimator, TransformerMixin):

    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column].as_matrix()#.reshape((-1,1))#.values.astype('U')
    
class DataFrameColumnExtracter_doc(BaseEstimator, TransformerMixin):

    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column].values.astype('U')

In [ ]:
### Perform grid search over the parameters of the model

# Parameters of the grid search
tuning_keys = ['clf__C', 'clf__tol']
tuned_parameters = {'clf__C': [1e-4, 1e-2, 1, 10], 'clf__tol': [1, 1e-2, 1e-4]} 

# Load the model # Logistic Regression model
clf_LR = Pipeline([# Sentences
                  ('sentences_features', Pipeline([
                      ('sentence_extractor', DataFrameColumnExtracter_doc('comment')),#.values.astype('U'),
                    ('vect', CountVectorizer(max_features = 1500, ngram_range = (1,5), analyzer = 'char')),
                     ('tf', TfidfTransformer(norm = 'l2'))
                  ])),
            # Classifier
            ('clf', LogisticRegression())#C=LR_C, tol=LR_C_tol))
        ])

# Perform grid search


clf = GridSearchCV(clf_LR, tuned_parameters, cv=5, verbose=0)


In [ ]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
nb_data = 5000
#best_model = clf.fit(dataset_train_comments[0:nb_data], dataset_train_comments['MV'][0:nb_data])
#best_parameters = best_model.best_params_  
#print(best_parameters)  
#best_result = best_model.best_score_  
#print(best_result)  
#save_obj(best_parameters, 'best_param_LR_aggregated')

In [ ]:
best_parameters = load_obj('best_param_LR_aggregated')
clf_LR.set_params(**best_parameters)
clf_LR.fit(dataset_train_comments[0:nb_data], dataset_train_comments['MV'][0:nb_data])

c:\users\agathebalayn\documents\thesis_related\virtualenvs\aif306_v2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('sentences_features', Pipeline(memory=None,
     steps=[('sentence_extractor', DataFrameColumnExtracter_doc(column='comment')), ('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lower...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [ ]:
### Evaluate general performance
train_pred = clf_LR.predict(dataset_train_comments[0:nb_data])
test_pred = clf_LR.predict(dataset_test_comments[0:nb_data])
print("Training accuracy: ", sk_met.accuracy_score(dataset_train_comments['MV'][0:nb_data], train_pred))
print("Test accuracy: ", sk_met.accuracy_score(dataset_test_comments['MV'][0:nb_data], test_pred))

C_train = sk_met.confusion_matrix(dataset_train_comments['MV'][0:nb_data], train_pred)
C_train = C_train / C_train.astype(np.float).sum(axis=0)
C_test = sk_met.confusion_matrix(dataset_test_comments['MV'][0:nb_data], test_pred)
C_test = C_test / C_test.astype(np.float).sum(axis=0)
print("Training confusion matrix:", C_train)
print("Test confusion matrix:", C_test)

Training accuracy:  0.9412
Test accuracy:  0.916
Training confusion matrix: [[0.94121419 0.05900621]
 [0.05878581 0.94099379]]
Test confusion matrix: [[0.92162677 0.17921147]
 [0.07837323 0.82078853]]


In [ ]:
import collections
collections.Counter(dataset_train_comments['MV'][0:nb_data])

Counter({0: 4422, 1: 578})

## Compute the fairness performance (both on training and test datasets)

In [ ]:
# The fairness performance have to be computed on the original ToxicityDataset folds.

# The dataset might be too large for the classifier to process all the data. In this case, it is splitted to get all the values.
prediction_col = "pred_1"
dataset_orig_train.dataset[prediction_col] = -1
dataset_orig_test.dataset[prediction_col] = -1
nb_data = 5000

def compute_pred_dataset(dataset_orig_train, prediction_col, nb_data):
    for i in range(int(len(dataset_orig_train.dataset) / nb_data)):
        low_interval = i*nb_data
        high_interval = (i+1)*nb_data
        dataset_orig_train.dataset[prediction_col].iloc[low_interval:high_interval] = clf_LR.predict(dataset_orig_train.dataset.iloc[low_interval:high_interval])
    dataset_orig_train.dataset[prediction_col].iloc[high_interval:] = clf_LR.predict(dataset_orig_train.dataset.iloc[high_interval:])
    return dataset_orig_train
dataset_orig_train = compute_pred_dataset(dataset_orig_train, prediction_col, nb_data)
dataset_orig_train.dataset[prediction_col].value_counts() 
    
    
    
print(dataset_orig_test.dataset[prediction_col].value_counts())

dataset_orig_test = compute_pred_dataset(dataset_orig_test, prediction_col, nb_data)
print(dataset_orig_test.dataset[prediction_col].value_counts())
print(len(dataset_orig_test.dataset))

#train_pred = clf_LR.predict(dataset_train_comments[0:nb_data])
#test_pred = clf_LR.predict(dataset_test_comments[0:nb_data])
#dataset_train_comments["pred_1"] = train_pred
#dataset_test_comments["pred_1"] = test_pred

c:\users\agathebalayn\documents\thesis_related\virtualenvs\aif306_v2\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
### Evaluate the fairness of the model
metrics_inclusiveness = InclusivenessLabelDatasetMetric(dataset_orig_test)

In [ ]:
metrics_inclusiveness.compute_metric(prediction_col)